## 1. Importing Libraries

In [1]:
import os

import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split

## 2. Reading the data

In [2]:
PROJECT_DIR = r"C:\Users\sneha\DADB\Flight-prediction-sagemaker"
DATA_DIR = "data"

In [3]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name) # It's help you to work with your local directory , then this above path make sense
    return pd.read_csv(file_path)

In [4]:
os.path.join(PROJECT_DIR, DATA_DIR,"flight_price")

'C:\\Users\\sneha\\DADB\\Flight-prediction-sagemaker\\data\\flight_price'

In [5]:
flights = get_data('flight_price')
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [6]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10683 rows
- Column `Route` and  `Total_Stops`
- Some of Columns not having proper data types

## 3.Preliminary Analysis

### 3.1 Check data types

In [7]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [8]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

### 3.2 Delete Duplicates

In [9]:
flights.duplicated().sum()

220

(
    flights
    .loc[flights.duplicated(keep=False)]
    .sort_values(['Airline','Date_of_Journey','Source','Destination','Route'])
)

### 3.3 Observation

- Type of `Date_of_Journey`,`Dep_Time`,`Arrival_Time` should be changed in Datetime
- Type of `Duration` and `Total_Stops` should be int
- There are 220 duplicates , this should be removed.


## 4. Detailed Analysis

## Airline

In [10]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [11]:
flights['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [12]:
(
flights
.Airline
.str.replace(" Premium economy","")
.str.replace(" Business","")
.str.title()
.unique()    
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

- Some of airline name are repeated but by different name.

## Date_of_Journey

In [13]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

## Source

In [14]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

## Destination

In [15]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

## Dept_Time

In [16]:
(
    flights
    .Dep_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [17]:
pd.to_datetime(flights.Dep_Time,format='%H:%M').dt.strftime('%H:%M').dtypes

dtype('O')

## Arrival_Time

In [18]:
(
    flights
    .Arrival_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]
    .str.split(" ",n=1)
    .str.get(1)
)

0        22 Mar
2        10 Jun
6        13 Mar
7        02 Mar
8        13 Mar
          ...  
10666    13 Jun
10667    13 Mar
10672    28 Jun
10673    28 May
10674    13 Mar
Name: Arrival_Time, Length: 4335, dtype: object

## Duration

In [19]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [20]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
)

2        19h
18       23h
33       22h
44       12h
53        3h
        ... 
10591    23h
10638    14h
10639    38h
10673    15h
10680     3h
Name: Duration, Length: 1031, dtype: object

In [21]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
    # .unique()
    # .drop(index=[])
)

6474    5m
Name: Duration, dtype: object

In [22]:
(
    flights
    .Duration
    .drop(index=[6474])
    .str.split(" ",expand=True)
    .set_axis(['hour',"minute"],axis=1)
    .assign(
        hour = lambda _df:(
               _df
               .hour
               .str.replace("h","")
               .astype(int)
			   .mul(60)
        ),
        minute = lambda _df: (
            _df
            .minute
            .str.replace("m", "")
			.fillna("0")
			.astype(int)
        )
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [23]:
(
	flights
	.Duration
	.drop(index=6474)
	.str.split(" ", expand=True)
	.set_axis(["hour", "minute"], axis=1)
	.assign(
		hour=lambda df_: (
			df_
			.hour
			.str.replace("h", "")
			.astype(int)
			.mul(60)
		),
		minute=lambda df_: (
			df_
			.minute
			.str.replace("m", "")
			.fillna("0")
			.astype(int)
		)
	)
	.sum(axis=1)
	.rename("duration_minutes")
	.to_frame()
	.join(flights.Duration.drop(index=6474))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


## Total Stops


In [24]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [25]:
(
	flights
	.Total_Stops
	.replace("non-stop", "0")
	.str.replace(" stops?", "", regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))
    .unique()
)                 

array([ 0.,  2.,  1.,  3., nan,  4.])

## Additional_info

In [26]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

## 5. Cleaning Operation

In [27]:
def convert_to_minutes(ser):
	return (
		ser
		.str.split(" ", expand=True)
		.set_axis(["hour", "minute"], axis=1)
		.assign(
			hour=lambda df_: (
				df_
				.hour
				.str.replace("h", "")
				.astype(int)
				.mul(60)
			),
			minute=lambda df_: (
				df_
				.minute
				.str.replace("m", "")
				.fillna("0")
				.astype(int)
			)
		)
		.sum(axis=1)
	)

In [28]:
def clean_data(df):
    return (
        df
        .drop(index=[6474])
		.drop_duplicates()
        .assign(**{
          col:df[col].str.strip()
          for col in df.select_dtypes(include="O").columns  
        })
        .rename(columns = str.lower)
        .assign(
            airline = lambda df_:(
                df_
                .airline
                .str.replace(" Premium economy","")
                .str.replace(" Business","")
                .str.title()
            ),
            date_of_journey = lambda df_: pd.to_datetime(df_.date_of_journey,dayfirst=True) ,
            dep_time = lambda df_ : pd.to_datetime(df_.dep_time,format='%H:%M').dt.time ,
            arrival_time = lambda df_ : pd.to_datetime(df_.arrival_time).dt.time,
            duration=lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_: (
				df_
				.total_stops
				.replace("non-stop", "0")
				.str.replace(" stops?", "", regex=True)
				.pipe(lambda ser: pd.to_numeric(ser))
			),
            additional_info=lambda df_: df_.additional_info.replace("No info", "No Info")
        )
       .drop(columns="route")
    )

In [29]:
clean_data(flights)

C:\Users\sneha\AppData\Local\Temp\ipykernel_12932\585260749.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda df_ : pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


In [30]:
# [col.lower() for col in flights.columns]

In [31]:
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [33]:
flights_cleaned = clean_data(flights)
flights_cleaned

C:\Users\sneha\AppData\Local\Temp\ipykernel_12932\585260749.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda df_ : pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


<h1>6. Split the Data</h1>

In [34]:
flights_final = flights_cleaned.sample(1000)

In [36]:
X = flights_final.drop(columns="price")
y = flights_final.price.copy()

In [37]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


<h1>7. Export the Subset</h1>

In [39]:
def export_data(X,y,name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR,file_name)
    X.join(y).to_csv(file_path, index=False)

    return pd.read_csv(file_path).head()

In [40]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Spicejet,2019-05-15,Kolkata,Banglore,15:55:00,21:25:00,330,1.0,No Info,7804
1,Jet Airways,2019-05-18,Delhi,Cochin,13:25:00,12:35:00,1390,2.0,No Info,15129
2,Indigo,2019-06-06,Delhi,Cochin,23:30:00,04:35:00,305,1.0,No Info,10112
3,Jet Airways,2019-05-01,Kolkata,Banglore,14:05:00,08:15:00,1090,1.0,In-flight meal not included,8586
4,Jet Airways,2019-04-06,Banglore,Delhi,07:10:00,10:10:00,180,0.0,In-flight meal not included,4544


In [41]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Vistara,2019-03-06,Mumbai,Hyderabad,21:55:00,16:45:00,1130,1.0,No Info,12395
1,Jet Airways,2019-06-12,Delhi,Cochin,09:35:00,12:35:00,1620,2.0,No Info,13292
2,Jet Airways,2019-06-12,Delhi,Cochin,18:25:00,12:35:00,1090,2.0,No Info,14237
3,Air India,2019-05-18,Kolkata,Banglore,08:20:00,21:50:00,810,2.0,No Info,10697
4,Jet Airways,2019-03-12,Banglore,New Delhi,08:55:00,10:25:00,1530,1.0,In-flight meal not included,11087


In [42]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-06-03,Delhi,Cochin,08:00:00,19:15:00,675,1.0,No Info,15058
1,Goair,2019-05-01,Delhi,Cochin,07:25:00,13:35:00,370,1.0,No Info,5842
2,Spicejet,2019-04-12,Kolkata,Banglore,06:55:00,09:30:00,155,0.0,No check-in baggage included,3841
3,Multiple Carriers,2019-03-06,Delhi,Cochin,07:00:00,18:50:00,710,1.0,No Info,15867
4,Air India,2019-04-09,Delhi,Cochin,05:55:00,07:40:00,1545,2.0,No Info,10021
